In [1]:
import theano
import theano.tensor as T
import numpy as np
import cv2
import sys
from loader import TieLoader

Couldn't import dot_parser, loading of dot files will not be possible.


Using cuDNN version 5105 on context None
Mapped name None to device cuda0: GeForce GTX TITAN X (0000:06:00.0)


In [2]:
from easydict import EasyDict
cfg = EasyDict()
cfg.SEQ_LENGTH = 250
cfg.TILE_SIZE = 32
cfg.OUT_SIZE = 1
cfg.TRAIN = EasyDict()
cfg.TRAIN.EPOCH = 0
cfg.TRAIN.EPOCH_SIZE = 1000
cfg.TRAIN.EPOCH_NUM = 20
cfg.gm_num = 4
cfg.ndim = 12
cfg.NAME = 'unet_true_grad%i'%(cfg.ndim)#'conv_net_no_bn_ndim12010'#

In [3]:

train_loader = TieLoader('data/ties32',
                         0.2,0.3,t_size=32,mask_size=cfg.OUT_SIZE,sample_size=cfg.TILE_SIZE,cache_samples=True)
test_loader = TieLoader('data/test_ties32',
                        -0.1,0.45,t_size=32,mask_size=cfg.OUT_SIZE,sample_size=cfg.TILE_SIZE,cache_samples=True)

In [4]:
# from dataset_tools import draw
# import matplotlib.pyplot as plt
# %matplotlib inline
# for x,y in data_generator(test_loader,epoch_size=1,shuffle=False):
#     print x.shape
#     print np.ones_like(y)[y<0.1].sum(),np.ones_like(y)[y>0.9].sum()
#     x = np.transpose(x,(0,2,3,1)).astype(np.uint8)
#     y = (y*255.).astype(np.uint8)
#     _y = np.zeros(x.shape[:3],dtype=np.uint8)
    
#     _y[:] = y[:,0,0][:,None,None]
#     draw(x,_y)

# Build model

In [5]:
from lasagne import layers as L
from lasagne.nonlinearities import rectify,softmax
from utils import NormedDense,L2NormLayer
from utils import get_network_str,load_weights
import lasagne
import denseNet
import uNet
from networks import baseline_norm,conv4_net,conv4_net_dense


def FCN(data,ndim,verbose=True,model_name='',input_shape = (None,3,None,None),pad='same',logger=None):
    datal = res = L.InputLayer(input_shape
                           ,data
                           ,name='data')
    res = uNet.build(datal)
    res = L.Conv2DLayer(res, ndim-1, 1, nonlinearity=None)
    res.W.name = 'outW'
    res.b.name = 'outb'
    res = L.DimshuffleLayer(res,(0,2,3,1),name='transpose')
    res = L2NormLayer(res,1e-8,name='l2norm')
    if(model_name!=''):
        print 'load model '+'models/'+model_name
        load_weights(res,'models/'+model_name)
    print get_network_str(res,incomings=True,outgoings=True)
    if(not (logger is None)):        
        logger.write(get_network_str(res,incomings=True,outgoings=True)+'\n')
        logger.flush()
    return res
        

data = T.tensor4(name='data')
label = T.tensor3(name='label')
net = FCN(data,ndim=cfg.ndim,model_name='%s%03d'%(cfg.NAME,cfg.TRAIN.EPOCH) if cfg.TRAIN.EPOCH > 0 else '',
          input_shape = (None,3,cfg.TILE_SIZE,cfg.TILE_SIZE),
          pad = 'same')

load pretrained UNet_params_pretrained

In -->      Layer    --> Out     Description                 
-------     -----    -------     -----------                 
[]          0        [1]         input(None, 3, 32, 32)      
[0]         1        [2]         None(None, 16, 32, 32)      
[1]         2        [3]         None(None, 16, 32, 32)      
[2]         3        [4]         contr_1_1(None, 16, 32, 32) 
[3]         4        [5]         None(None, 16, 32, 32)      
[4]         5        [6]         None(None, 16, 32, 32)      
[5]         6        [7, 68]     contr_1_2(None, 16, 32, 32) 
[6]         7        [8]         pool1(None, 16, 16, 16)     
[7]         8        [9]         None(None, 32, 16, 16)      
[8]         9        [10]        None(None, 32, 16, 16)      
[9]         10       [11]        contr_2_1(None, 32, 16, 16) 
[10]        11       [12]        None(None, 32, 16, 16)      
[11]        12       [13]        None(None, 32, 16, 16)      
[12]        13       [14, 58] 

# Train

In [ ]:
from gmm_op import get_gmm,calc_log_prob_gmm
from theano_utils import split,histogram_loss,split_tr_p_n

def make_classifier(X,label,non_learn_params):
    X = X.reshape((-1,X.shape[-1]))
    x_tr,x_p,x_n = split_tr_p_n(X,label.flatten())
    m,c,w = get_gmm(x_tr,cfg.gm_num,cfg.ndim,use_approx_grad=False)
    p_n = calc_log_prob_gmm(x_n,m,c,w)
    p_p = calc_log_prob_gmm(x_p,m,c,w)
    loss = histogram_loss(p_n,p_p,
                          non_learn_params['min_cov'],
                          100,
                          non_learn_params['width'])[0]
    prediction = T.nnet.sigmoid(T.concatenate([p_p,p_n],axis=0))
    Y = T.concatenate([T.ones_like(p_p),T.zeros_like(p_n)],axis=0)
    return loss,X,Y,prediction,m,c,w,p_p,p_n

def make_train(net,data,label,non_learn_params):
    sym = L.get_output(net ,deterministic=False)
    s = int((L.get_output_shape(net)[1]-1)/2)
    sym = sym[:,s:s+1,s:s+1,:]
    loss,X,Y,prediction,m,c,w,p_p,p_n = make_classifier(sym,label,non_learn_params)
    params = L.get_all_params(net,trainable=True)
    updates = lasagne.updates.adam(loss,params,non_learn_params['lr'])
    return theano.function([data, label], [loss,X,Y,prediction,m,c,w],\
                               allow_input_downcast=True, updates=updates)


def make_test(net,data,label,non_learn_params):
    sym = L.get_output(net ,deterministic=True)
    s = int((L.get_output_shape(net)[1]-1)/2)
    sym = sym[:,s:s+1,s:s+1]
    loss,X,Y,prediction,m,c,w,p_p,p_n = make_classifier(sym,label,non_learn_params)
    return theano.function([data, label], [loss,X,Y,prediction,m,c,w],\
                               allow_input_downcast=True)
        

non_learn_params={'min_cov' : theano.shared(1e-3),
                  'lr' : theano.shared(np.array(1e-2, dtype=theano.config.floatX)),
                  'width': theano.shared(4.)}

train_fn = make_train(net,data,label,non_learn_params)
print 'train_fn compiled'
test_fn = make_test(net,data,label,non_learn_params)
print 'test_fn compiled'

/usr/local/lib/python2.7/dist-packages/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode=self.mode,
/usr/local/lib/python2.7/dist-packages/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode=self.mode,
/usr/local/lib/python2.7/dist-packages/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode=self.mode,


train_fn compiled
test_fn compiled


In [ ]:
from sklearn.metrics import average_precision_score
import datetime
from utils import get_pp_pn
import os
from train_tools import fit
from test_tools import test_network

metrix = { 'aps' : average_precision_score,
           'pp'  : lambda l,pred : np.ones(len(l))[(pred < 0.5)&(l<0.1)].sum()/np.ones(len(l))[l<0.1].sum(),
           'pn'  : lambda l,pred : np.ones(len(l))[(pred >= .5)&(l>0.9)].sum()/np.ones(len(l))[l>0.9].sum(),
           'int_pp_pn' : lambda l,pred : get_pp_pn(l,pred)}

def update_params(epoch,params):
    if(epoch == 0):
        params['min_cov'].set_value(1e-4)
        params['lr'].set_value(5e-2)
    if(epoch == 4):
        params['min_cov'].set_value(1e-8)
        params['lr'].set_value(5e-3)
    if(epoch == 10):
        params['lr'].set_value(1e-3)
        
fout = open("out.txt",'a')
fout.write('################### train network '+cfg.NAME+ ' ' + str(datetime.datetime.now())+'################\n')    
fout.write(str(cfg))
fit(cfg.NAME,net,train_fn,test_fn,train_loader,test_loader,non_learn_params,
    update_params=update_params,
    metrix = metrix,
    logger=fout,
    train_esize = 1500,
    test_esize = 750,
    epochs=cfg.TRAIN.EPOCH_NUM-cfg.TRAIN.EPOCH)

test_network(cfg.NAME,FCN,cfg.ndim,cfg.TRAIN.EPOCH_NUM-1,cfg.gm_num)
fout.write('################### done #######################\n')    

checkpoint models/unet_true_grad12000.npz
train


In [23]:
non_learn_params['lr'].set_value(0.001)

In [25]:
for i,batch in enumerate(data_generator(test_loader,epoch_size=100,shuffle=False)):
    res = test_fn(*batch)
    if(i == 0):
        p =res[-1]
        n = res[-2]
    p = np.concatenate((p,res[-1]))
    n = np.concatenate((n,res[-2]))
plt.hist(np.clip(res[-1],-10,100),100,normed=True)
plt.hist(np.clip(res[-2],-10,100),100,normed=True)
plt.show()  
plt.hist(np.clip(p,-20,100),100,normed=True)
plt.hist(np.clip(n,-20,100),100,normed=True)
plt.show()

In [12]:
import os
import cv2
import numpy as np
#from gmm_bg_substructor import BgSubstructor
from sklearn import mixture
from sklearn.metrics import average_precision_score
from dataset_tools import *
from gmm_op import get_gmm,calc_log_prob_gmm
import theano.tensor as T
from lasagne import layers as L
import theano

def soft_predict_sym(features,means,covars,weights):
    return 1.-T.nnet.sigmoid(calc_log_prob_gmm(features,means,covars,weights))


def make_features(feature_fn,imgs):
    data = None
    for i in range(len(imgs)):
        tmp = feature_fn(np.transpose(imgs[i:i+1],(0,3,1,2)).astype(np.float32))[0]
        if(data is None):
            data = np.empty((len(imgs),)+tmp.shape,dtype=np.float32)
        data[i] = tmp
    return data

def make_gmms(shape,gm_num):
    gmms = []
    for i in range(shape[0]):
        for j in range(shape[1]):
            gmms.append(mixture.GaussianMixture(covariance_type='diag',
                               n_components=gm_num,
                               max_iter=1000,
                               warm_start=True))
    return gmms

    
def fit_gmms(features,gmms,masks = None,out_masks=None):
    print features.shape
    for i in range(features.shape[1]):
        for j in range(features.shape[2]):
            f = features[:,i,j]
            gmm = gmms[i*features.shape[2]+j]
            if not (masks is None):
                if(len(f[masks[:,i,j] < 30]) > gmm.n_components*3):
                    f = f[masks[:,i,j] < 30]
            
            if not (masks is None):
                m = masks[:,i,j] 
                if(m[(m > 30) & (m< 240)].size > 0.9*m.size):
                    out_masks[:,i,j] = 128
                    continue
                else:
                    gmm.fit(f)
            else:
                gmm.fit(f)

def predict_pixelwise(features,gmms,predict_fn,masks):
    res = np.zeros_like(features[:,:,:,0])
    for i in range(features.shape[1]):
        for j in range(features.shape[2]):
            m = masks[:,i,j] 
            if(m[(m > 30) & (m< 240)].size > 0.9*m.size):
                continue
            gmm = gmms[i*features.shape[2]+j]
            res[:,i,j] = predict_fn(features[:,i,j,:],gmm.means_,gmm.covariances_,gmm.weights_)
    return res



def calc_aps(pred,masks):
    masks = masks.flatten()
    pred = pred.flatten()
    m = np.zeros_like(masks)
    m[masks > 240] = 1.
    m = m[(masks<30) | (masks > 240)]
    pred = pred[(masks<30) | (masks > 240)]
    return average_precision_score(m,1.-pred)

def make_test_as_train(feature_fn,predict_fn,
                       gm_num,
                      out_dir,
                      dataset='dataset',
                      max_frames=400,
                      im_size = (320,240),
                      train_size = 100):
    try:
        os.mkdir(out_dir)
    except:
        pass
    aps_log = open(out_dir+'/aps.txt','w')
    for in_dir,out_dir in iterate_folders(dataset,out_dir):
        print 'run in ',in_dir
        for i,(imgs,masks) in enumerate(iterate_bathced(in_dir,max_frames,im_size)):
            if((masks[(masks>30) & (masks < 240)].size > 0.9*masks.size) or
               (masks[(masks>240)].size < 10)):
                print 'skip' 
                continue
            print in_dir,'generate_features,',
            data = make_features(feature_fn,imgs)
            gmms = make_gmms(imgs.shape[1:-1],gm_num)
            print 'fit gmms,',
            fit_gmms(data[:train_size],gmms,masks[:train_size],masks[train_size:])
            print 'predict,',
            prediction = predict_pixelwise(data[train_size:],gmms,predict_fn,masks[train_size:])
            aps = calc_aps(prediction,masks[train_size:])
            print in_dir,'aps = ',aps
            aps_log.write(in_dir+' : '+str(aps)+'\n')
            aps_log.flush()
            print 'save'
            imgs = imgs[train_size:]
            masks = masks[train_size:]      
            prediction = (prediction*255).astype(np.uint8)
            for i in range(len(imgs)):
                cv2.imwrite(out_dir+'/'+str(i)+'.png',prediction[i])
                cv2.imwrite(out_dir+'/'+str(i)+'_true.png',masks[i])
                cv2.imwrite(out_dir+'/'+str(i)+'_input.jpg',imgs[i])
            break
        print ''
    print 'test complete'

def test_network(name,network,ndim,epoch,gm_num):
    

In [ ]:
test_network(cfg.NAME,FCN,cfg.ndim,cfg.TRAIN.EPOCH_NUM-1,cfg.gm_num)

load pretrained UNet_params_pretrained
load model models/unet12019

In -->      Layer    --> Out     Description                  
-------     -----    -------     -----------                  
[]          0        [1]         input(500, 3, 240, 320)      
[0]         1        [2]         None(500, 16, 240, 320)      
[1]         2        [3]         None(500, 16, 240, 320)      
[2]         3        [4]         contr_1_1(500, 16, 240, 320) 
[3]         4        [5]         None(500, 16, 240, 320)      
[4]         5        [6]         None(500, 16, 240, 320)      
[5]         6        [7, 68]     contr_1_2(500, 16, 240, 320) 
[6]         7        [8]         pool1(500, 16, 120, 160)     
[7]         8        [9]         None(500, 32, 120, 160)      
[8]         9        [10]        None(500, 32, 120, 160)      
[9]         10       [11]        contr_2_1(500, 32, 120, 160) 
[10]        11       [12]        None(500, 32, 120, 160)      
[11]        12       [13]        None(500, 32, 120